#### 카드 짝 맞추기
---
https://school.programmers.co.kr/learn/courses/30/lessons/72415

1. 여러 카드 조합 순서별로 모든 경우의 수 탐색
2. 한 순서에 대해서
    - 이전 카드 제거 위치 인 두개의 경우에서 이번에 지울 카드 위치 에 대해 각 최소 입력 수 계산 및 저장 (dp)

In [96]:
import copy
class Card:
    def __init__(self, num):
        self.num=num
        self.left = 0
        self.right = 3
        self.up = 0
        self.down = 3
    def retrun_connection(self):
        return self.up, self.down, self.left, self.right 
class CardMap:
    loc_ = {n:[] for n in range(1,7)}
    card_map = []

    @classmethod 
    def init_map(cls,board):
        for i in range(4):
            CardMap.card_map.append([])
            left = 0
            for j in range(4):
                card = Card(board[i][j])
                card.left = left
                if board[i][j]:
                    for sub_j in range(left, j):
                        CardMap.card_map[i][sub_j].right = j
                    left = j
                    CardMap.loc_[board[i][j]].append([i,j])
                CardMap.card_map[i].append(card)

        for j in range(4):
            up = 0
            for i in range(4):
                CardMap.card_map[i][j].up = up
                if board[i][j]:
                    for sub_i in range(up, i):
                        CardMap.card_map[sub_i][j].down = i
                    up = i

        for n in range(1,7):
            if len(CardMap.loc_[n])==0:
                del CardMap.loc_[n]
                
    def __init__(self):
        self.map = copy.deepcopy(CardMap.card_map)

    def line_move(self, cur, goal):
        if cur == goal:
            return 0
        cnt =0
        cur_r, cur_c = cur
        if cur_r < goal[0]:
            while cur_r<goal[0]:
                if self.map[cur_r][cur_c].down >goal[0]:
                    cnt += goal[0]-cur_r
                    break
                cur_r = self.map[cur_r][cur_c].down
                cnt+=1
        elif cur_r > goal[0]:
            while cur_r>goal[0]:
                if self.map[cur_r][cur_c].up <goal[0]:
                    cnt += cur_r-goal[0]
                    break
                cur_r = self.map[cur_r][cur_c].up
                cnt+=1

        elif cur_c < goal[1]:
            while cur_c<goal[1]:
                if self.map[cur_r][cur_c].right >goal[1]:
                    cnt += goal[1]-cur_c
                    break
                cur_c = self.map[cur_r][cur_c].right
                cnt+=1
        else:
            while cur_c>goal[1]:
                if self.map[cur_r][cur_c].left < goal[1]:
                    cnt += cur_c-goal[1]
                    break
                cur_c = self.map[cur_r][cur_c].left
                cnt+=1

        return cnt

    def move(self, cur, goal):
        cnt1 = self.line_move(cur,[cur[0],goal[1]])+self.line_move([cur[0],goal[1]],goal)
        cnt2 = self.line_move(cur,[goal[0],cur[1]])+self.line_move([goal[0],cur[1]],goal)
        return min(cnt1, cnt2)

    def delete(self, num):
        locs = CardMap.loc_[num]
        for r,c in locs:
            card = self.map[r][c]
            u_, d_, l_, r_ = card.retrun_connection()
            card.num = 0
            for sub_r in range(u_,r):
                self.map[sub_r][c].down = d_
            for sub_r in range(r+1, d_+1):
                self.map[sub_r][c].up = u_
            for sub_c in range(l_,c):
                self.map[r][sub_c].right = r_
            for sub_c in range(c+1, r_+1):
                self.map[r][sub_c].left = l_

    def show(self):
        for cs in self.map:
            print([[c.num,c.retrun_connection()] for c in cs])

    def calc(self,cur, card_order):
        # self.show()
        # print()
        dp = [[0,0]]
        order = card_order[0]
        card1, card2 = CardMap.loc_[order]
        cur1, cur2 = cur, cur
        cnt1 = min(dp[0][0]+self.move(cur1, card1)+self.move(card1, card2),
                   dp[0][1]+self.move(cur2, card1)+self.move(card1, card2))
        cnt2 = min(dp[0][0]+self.move(cur1, card2)+self.move(card2, card1),
                   dp[0][1]+self.move(cur2, card2)+self.move(card2, card1))
        dp.append([cnt2, cnt1])
        self.delete(order)
        cur1, cur2 = card1, card2

        for i in range(1, len(card_order)):
            # self.show()
            order = card_order[i]
            card1, card2 = CardMap.loc_[order]
            cnt1 = min(dp[i][0]+self.move(cur1, card1)+self.move(card1, card2),
                    dp[i][1]+self.move(cur2, card1)+self.move(card1, card2))
            cnt2 = min(dp[i][0]+self.move(cur1, card2)+self.move(card2, card1),
                    dp[i][1]+self.move(cur2, card2)+self.move(card2, card1))
            dp.append([cnt2, cnt1])
            self.delete(order)
            cur1, cur2 = card1, card2
        #     print()
        # self.show()
        return min(dp[-1])

import itertools

def solution(board, r, c):


    CardMap.init_map(board)
    cards = list(CardMap.loc_.keys())
    answer = 16*6

    for card_order in list(itertools.permutations(cards)):
        local_cardMap = CardMap()
        cnt = local_cardMap.calc([r,c],card_order)
        if cnt< answer:
            answer = cnt
    # local_cardMap = CardMap()
    # cnt = local_cardMap.calc([r,c],[1,4,2,3])

    return answer+2*len(cards)

In [97]:
solution([[3, 0, 0, 2], [0, 4, 1, 0], [0, 1, 4, 0], [2, 0, 0, 3]], 0, 1)

21